### Importing the libraries

In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.base import TransformerMixin
from sklearn.linear_model import Lasso
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,BernoulliNB, MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from mlxtend.classifier import StackingClassifier
from sklearn.ensemble import VotingClassifier
from joblib import dump, load
from sklearn.pipeline import Pipeline

%matplotlib inline

### Importing the dataset

In [56]:
dataset = pd.read_csv("creditcard.csv")
dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [57]:
dataset.corr()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
Time,1.000000,1.173963e-01,-1.059333e-02,-4.196182e-01,-1.052602e-01,1.730721e-01,-6.301647e-02,8.471437e-02,-3.694943e-02,-8.660434e-03,...,4.473573e-02,1.440591e-01,5.114236e-02,-1.618187e-02,-2.330828e-01,-4.140710e-02,-5.134591e-03,-9.412688e-03,-0.010596,-0.012323
V1,0.117396,1.000000e+00,4.697350e-17,-1.424390e-15,1.755316e-17,6.391162e-17,2.398071e-16,1.991550e-15,-9.490675e-17,2.169581e-16,...,-1.755072e-16,7.477367e-17,9.808705e-16,7.354269e-17,-9.805358e-16,-8.621897e-17,3.208233e-17,9.820892e-16,-0.227709,-0.101347
V2,-0.010593,4.697350e-17,1.000000e+00,2.512175e-16,-1.126388e-16,-2.039868e-16,5.024680e-16,3.966486e-16,-4.413984e-17,-5.728718e-17,...,8.444409e-17,2.500830e-16,1.059562e-16,-8.142354e-18,-4.261894e-17,2.601622e-16,-4.478472e-16,-3.676415e-16,-0.531409,0.091289
V3,-0.419618,-1.424390e-15,2.512175e-16,1.000000e+00,-3.416910e-16,-1.436514e-15,1.431581e-15,2.168574e-15,3.433113e-16,-4.233770e-16,...,-2.971969e-17,4.648259e-16,2.115206e-17,-9.351637e-17,4.771164e-16,6.521501e-16,6.239832e-16,7.726948e-16,-0.210880,-0.192961
V4,-0.105260,1.755316e-17,-1.126388e-16,-3.416910e-16,1.000000e+00,-1.940929e-15,-2.712659e-16,1.556330e-16,5.195643e-16,3.859585e-16,...,-9.976950e-17,2.099922e-16,6.002528e-17,2.229738e-16,5.394585e-16,-6.179751e-16,-6.403423e-17,-5.863664e-17,0.098732,0.133447
V5,0.173072,6.391162e-17,-2.039868e-16,-1.436514e-15,-1.940929e-15,1.000000e+00,7.926364e-16,-4.209851e-16,7.589187e-16,4.205206e-16,...,-1.368701e-16,5.060029e-16,1.637596e-16,-9.286095e-16,5.625102e-16,9.144690e-16,4.465960e-16,-3.299167e-16,-0.386356,-0.094974
V6,-0.063016,2.398071e-16,5.024680e-16,1.431581e-15,-2.712659e-16,7.926364e-16,1.000000e+00,1.429426e-16,-1.707421e-16,1.114447e-16,...,-1.575903e-16,-3.362902e-16,-7.232186e-17,-1.261867e-15,1.081933e-15,-2.378414e-16,-2.623818e-16,4.813155e-16,0.215981,-0.043643
V7,0.084714,1.991550e-15,3.966486e-16,2.168574e-15,1.556330e-16,-4.209851e-16,1.429426e-16,1.000000e+00,-8.691834e-17,7.933251e-16,...,1.938604e-16,-1.058131e-15,2.327911e-16,-2.589727e-17,1.174169e-15,-7.334507e-16,-5.886825e-16,-6.836764e-17,0.397311,-0.187257
V8,-0.036949,-9.490675e-17,-4.413984e-17,3.433113e-16,5.195643e-16,7.589187e-16,-1.707421e-16,-8.691834e-17,1.000000e+00,2.900829e-16,...,-2.412439e-16,5.475559e-16,3.897104e-16,-1.802967e-16,-1.390791e-16,-1.209975e-16,1.733633e-16,-4.484325e-16,-0.103079,0.019875
V9,-0.008660,2.169581e-16,-5.728718e-17,-4.233770e-16,3.859585e-16,4.205206e-16,1.114447e-16,7.933251e-16,2.900829e-16,1.000000e+00,...,4.578389e-17,2.871855e-17,5.929286e-16,-2.346385e-16,1.099645e-15,-1.388725e-15,-2.287414e-16,9.146779e-16,-0.044246,-0.097733


In [58]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
Time      284807 non-null float64
V1        284807 non-null float64
V2        284807 non-null float64
V3        284807 non-null float64
V4        284807 non-null float64
V5        284807 non-null float64
V6        284807 non-null float64
V7        284807 non-null float64
V8        284807 non-null float64
V9        284807 non-null float64
V10       284807 non-null float64
V11       284807 non-null float64
V12       284807 non-null float64
V13       284807 non-null float64
V14       284807 non-null float64
V15       284807 non-null float64
V16       284807 non-null float64
V17       284807 non-null float64
V18       284807 non-null float64
V19       284807 non-null float64
V20       284807 non-null float64
V21       284807 non-null float64
V22       284807 non-null float64
V23       284807 non-null float64
V24       284807 non-null float64
V25       284807 non-null float64
V26  

In [59]:
features = dataset.iloc[:,:-1]
# features = features.drop(columns=["V22","V23","V24"])
goal = dataset.iloc[:,-1]
features.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


In [113]:
class preprocessing(TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, dataset ,*args, **kwargs):
#         dataset = dataset.drop(columns = ["Time"])
        self.stscalers = []
        
        for col in dataset.columns:
            stscaler = StandardScaler()
            stscaler.fit(dataset.loc[:,col].values.reshape(-1,1))
            self.stscalers.append(stscaler)
        
        return self
            
    def transform(self , dataset):
        self.newdataset = pd.DataFrame()
#         dataset = dataset.drop(columns = ["Time"])
        
        for stscaler,col in zip(self.stscalers,dataset.columns):
            feature = stscaler.transform(dataset[col].values.reshape(-1,1))
            self.newdataset[col] = np.reshape(feature, (-1))
        
        return self.newdataset
            
        

In [114]:
def plot(feature = [], actual = [], predicted = [], alpha = 1):
    plt.plot(feature, actual, '.', feature, predicted, '*')
    plt.title('alpha = {}'.format(alpha))
    plt.show()

In [115]:
preproces = preprocessing()

preproces.fit(features)
new_features = preproces.transform(features)

### Splitting the dataset

In [69]:
train_set, test_set, goal_train, goal_test = train_test_split(new_features,goal,test_size =0.2,random_state=0)

### Choosing the best ML Classifiers
- K-Nearest Neighbors.
- Support Vector Machines.
- Decision Tree Classifiers/Random Forests.
- Naive Bayes.
- Linear Discriminant Analysis.
- Logistic Regression.

In [10]:
knn = KNeighborsClassifier(n_neighbors = 5)
ksvc = SVC(kernel ='rbf', random_state = 0)
lsvc = LinearSVC(random_state = 0)
dt = DecisionTreeClassifier(min_samples_leaf=5, max_depth=3, random_state=0)
gnb = GaussianNB()
bnb = BernoulliNB()
lda = LDA(n_components=1)
rf = RandomForestClassifier(max_depth=2, random_state=0)
lr = LogisticRegression(random_state = 0)

In [11]:
for clf, label in zip([knn,ksvc,lsvc,dt,lda], ['KNN', 'KernelSVC', 'LinearSVC', 'DecisionTree', 'LDA']):
    clf.fit(train_set,goal_train)
    goal_predict = clf.predict(test_set)
    print(label)
    print(classification_report(goal_test,goal_predict))

KNN
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56861
           1       0.89      0.80      0.84       101

    accuracy                           1.00     56962
   macro avg       0.94      0.90      0.92     56962
weighted avg       1.00      1.00      1.00     56962

KernelSVC
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56861
           1       0.93      0.66      0.77       101

    accuracy                           1.00     56962
   macro avg       0.96      0.83      0.89     56962
weighted avg       1.00      1.00      1.00     56962



C:\Users\clt\Anaconda3\envs\ML diploma\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56861
           1       0.88      0.71      0.79       101

    accuracy                           1.00     56962
   macro avg       0.94      0.86      0.89     56962
weighted avg       1.00      1.00      1.00     56962

DecisionTree
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56861
           1       0.80      0.75      0.78       101

    accuracy                           1.00     56962
   macro avg       0.90      0.88      0.89     56962
weighted avg       1.00      1.00      1.00     56962

LDA
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56861
           1       0.88      0.76      0.81       101

    accuracy                           1.00     56962
   macro avg       0.94      0.88      0.91     56962
weighted avg       1.00      1.00      1.00    

- we can see that the best classifiers are KNNs , kernelsvc , linear svc , decision trees and LDA
- we will try to increase their accuracies using grid search 

In [12]:
for clf,label in zip([knn, ksvc, lsvc, dt, lda],['KNN', 'KernelSVC', 'LinearSVC', 'DecisionTree', 'LDA']):
    if label is 'KNN':
        parameters = [{"n_neighbors": [5,10,15]}]
        grid_search = GridSearchCV(estimator = clf,
                           param_grid = parameters,
                           cv = 5,
                           n_jobs = -1)
        grid_search = grid_search.fit(train_set, goal_train)
        print("best accuracy is :" , grid_search.best_score_)
        print(grid_search.best_params_)
    
    if label is 'KernelSVC':
        parameters = [{"C": [0.1,0.5,0.7,1]}]
        grid_search = GridSearchCV(estimator = clf,
                           param_grid = parameters,
                           cv = 5,
                           n_jobs = -1)
        grid_search = grid_search.fit(train_set, goal_train)
        print(label)
        print("best accuracy is :" , grid_search.best_score_)
        print(grid_search.best_params_)
    if label is 'LinearSVC':
        parameters = [{"penalty": ["l1",'l2'],
                      "loss": ['hinge', 'squared_hinge'],
                      "C": [0.1,0.5,1]}]
        grid_search = GridSearchCV(estimator = clf,
                           param_grid = parameters,
                           cv = 5,
                           n_jobs = -1)
        grid_search = grid_search.fit(train_set, goal_train)
        print(label)
        print("best accuracy is :" , grid_search.best_score_)
        print(grid_search.best_params_)
    if label is 'DecisionTree':
        parameters = [{"criterion": ["gini", "entropy"],
                      "max_depth": [10,15,20],
                      "min_samples_leaf": [5,8,10,15]}]
        grid_search = GridSearchCV(estimator = clf,
                           param_grid = parameters,
                           cv = 5,
                           n_jobs = -1)
        grid_search = grid_search.fit(train_set, goal_train)
        print(label)
        print("best accuracy is :" , grid_search.best_score_)
        print(grid_search.best_params_)
    if label is "LDA":
        parameters = [{"n_components": [1, 2, 3]}]
        grid_search = GridSearchCV(estimator = clf,
                           param_grid = parameters,
                           cv = 5,
                           n_jobs = -1)
        grid_search = grid_search.fit(train_set, goal_train)
        print(label)
        print("best accuracy is :" , grid_search.best_score_)
        print(grid_search.best_params_)



best accuracy is : 0.9994689372160899
{'n_neighbors': 5}
KernelSVC
best accuracy is : 0.9993372687572692
{'C': 1}


C:\Users\clt\Anaconda3\envs\ML diploma\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC
best accuracy is : 0.9993679913976606
{'C': 1, 'loss': 'hinge', 'penalty': 'l2'}
DecisionTree
best accuracy is : 0.9994294366784437
{'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 8}
LDA
best accuracy is : 0.99938554719217
{'n_components': 1}


### using the stacking classifier to combine all classifiers into one.

In [13]:
# knn = KNeighborsClassifier(n_neighbors = 5)
# ksvc_final = SVC(kernel ='rbf', random_state = 0 , C = 1)
# lsvc_final = LinearSVC(C = 1, loss = "hinge", penalty = "l2")
# dt_final = DecisionTreeClassifier(criterion = "gini", min_samples_leaf=8, max_depth=10)
# lda_final = LDA(n_components=1)
# lr = LogisticRegression(random_state = 0)
# sclf = StackingClassifier(classifiers=[ksvc_final, lsvc_final, dt_final, knn, lda_final], meta_classifier=lr)

# sclf.fit(train_set,goal_train)
# goal_predict = sclf.predict(test_set)
# print(classification_report(goal_test,goal_predict))

In [70]:
knn = KNeighborsClassifier(n_neighbors = 5)
ksvc_final = SVC(kernel ='rbf', random_state = 0 , C = 1)
lsvc_final = LinearSVC(C = 1, loss = "hinge", penalty = "l2")
dt_final = DecisionTreeClassifier(criterion = "gini", min_samples_leaf=8, max_depth=10)
lda_final = LDA(n_components=1)
# sclf = StackingClassifier(classifiers=[ksvc_final, lsvc_final, dt_final, knn], meta_classifier=lda_final)

eclf = VotingClassifier(estimators=[('knn', knn), ('ksvc', ksvc_final), ('lsvc', lsvc_final), ('dt', dt_final), ('lda', lda_final)], voting='hard')
eclf.fit(train_set,goal_train)
goal_predict = eclf.predict(test_set)
print(classification_report(goal_test,goal_predict))

C:\Users\clt\Anaconda3\envs\ML diploma\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56861
           1       0.88      0.82      0.85       101

    accuracy                           1.00     56962
   macro avg       0.94      0.91      0.93     56962
weighted avg       1.00      1.00      1.00     56962



### Creating a pipeline

In [152]:
pipe = Pipeline([('preprocessor', preprocessing()), ('classifier', eclf)])

In [153]:
pipe.fit(train_set,goal_train)
pipe.predict(train_set.iloc[:10,:])

C:\Users\clt\Anaconda3\envs\ML diploma\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [154]:
if __name__ == "__main__":
    dump(pipe, "creditcardfraudmodel.pkl", True)
    print('dumping complete')

dumping complete


### Now the model is ready for use in the main code